# Importando as biblitecas

In [ ]:
import pandas as pd
import re
import numpy as np

# Lendo o arquivo de mortalidade

In [ ]:
Mortalidade_2019 = pd.read_csv(r'C:\Projeto Final\data\Mortalidade_Geral_2019.csv', sep = ';')

# Estudando a tabela 

In [ ]:
Mortalidade_2019.head()

In [ ]:
Mortalidade_2019.info()

In [ ]:
Mortalidade_2019.columns

# Fazendo Verificação e Limpeza da Tabela

In [ ]:
Mortalidade_2019 = Mortalidade_2019.loc[:,['DTOBITO', 'CODMUNOCOR', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'CAUSABAS']]

Vendo os gêneros que mais sofreram com as mortes

In [ ]:
Mortalidade_2019['SEXO'].value_counts()

In [ ]:
#Ao verificar que o Sexo que está igual a 1 possui maior valor, pode-se confirmar que são os homens, pois ao pesquisar sobre a mortalidade de gêneros, os homens são os que mais sofrem mortes

Corrigindo a coluna de idade

In [ ]:
Mortalidade_2019['IDADE'] = Mortalidade_2019['IDADE'].astype(str)

def limpeza_idade(linha):
    x = linha['IDADE']
    if linha['DTOBITO']  == linha['DTNASC'] or linha['DTOBITO'] == linha['DTNASC']+1:
        return 0

    elif x.startswith('5') and len(x) == 3:
        x_new_5 = x.replace('5', '1', 1)
        x_new_5 = float(x)
        return x_new_5

    elif x.startswith('4') and len(x) == 3:
        x_new_4 = float(x[1:])
        return x_new_4

    elif x.startswith('3') and len(x) == 3 or len(x) == 2:
        x_new_3 = float(x[1:])
        x_new_3 = (x_new_3 * 833334/100)
        return x_new_3

    elif x.startswith('2') and len(x) == 3 or len(x) == 2:
        x_new_2 = float(x[1:])
        x_new_2 = (x_new_2 * 114155/10000)
        return x_new_2

    elif x.startswith('1') and len(x) == 3 or len(x) == 2:
        x_new_1 = float(x[1:])
        x_new_1 = (x_new_1 * 1.90259/1000000)
        return x_new_1

Mortalidade_2019['IDADE'] = Mortalidade_2019.apply(limpeza_idade, axis = 1)

Mudando o formato das datas da coluna de óbito e nascimento

In [ ]:
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].astype(str).str.zfill(8)                          # deixa todas as linhas com 8 números, alem de transformar em str
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].apply(lambda x: f'{x[0:2]}/{x[2:4]}/{x[4:]}')     # aplica '/' em todas as datas
Mortalidade_2019['DTOBITO'] = pd.to_datetime(Mortalidade_2019['DTOBITO'])                                   # transforma em datetime

In [ ]:
Mortalidade_2019['DTNASC'] = Mortalidade_2019['DTNASC'].fillna(0)                      #transforma nulo em zeros
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC']).dt.date        #transforma em data
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC'])

Arrumando a coluna de Causas e classificando cada uma das siglas 

In [ ]:
Mortalidade_2019['CAUSABAS'] = Mortalidade_2019['CAUSABAS'].apply(lambda x: f'{x[0:3]}')

In [ ]:
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['CAUSABAS']

In [ ]:
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[O-Q].*', 'problemas na gestão/Parto e malformações')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('R.*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[S-T].*', 'Consequências de causas externas (lesões/ traumatismos)')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[U-Y].*', 'Causas externas de morbidade e de mortalidade (acidente/agressao)')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('Z.*', 'Doença')

# Lendo arquivo CID para juntar as tabelas de CID e Mortalidade

In [ ]:
CID_2019 = pd.read_excel(r'C:\Projeto Final\data\CID_2019.xlsx', header=15, skipfooter=8)

# Fazendo o tratamento do arquivo CID

In [ ]:
CID_2019['Total'] = CID_2019['Total'].apply(lambda x: f'{x[0:3]}')
CID_2019 = CID_2019.loc[:,['Total']]
CID_2019  = CID_2019.drop(CID_2019.loc[CID_2019['Total']=='Cap'].index)
CID_2019['Classificação de causas'] = CID_2019['Total']

Classificação de causas (coluna CAUSABAS)

    - A00 ~ E99 --> doenças

    - F00       --> Transtornos Mentais

    - G00 ~ N99 --> doenças

    - O00 ~ Q99 --> problemas na gestão/Parto e malformações

    - R00       --> doenças

    - S00 ~ T98 --> Consequências de causas externas (lesões/ traumatismos)

    - U00 ~ Y98 --> Causas externas de morbidade e de mortalidade (acidente/agressao/intoxicação/falta de comida/falta de agua/queda/tiro)

    - Z00 ~     --> doenças

In [ ]:
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[O-Q].*', 'problemas na gestão/Parto e malformações')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('R.*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[S-T].*', 'Consequências de causas externas (lesões/ traumatismos)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[U-Y].*', 'Causas externas de morbidade e de mortalidade (acidente/agressao)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('Z.*', 'Doença')

# Lendo Arquivo de Municípios/Estados para juntar com a tabela de mortalidade

In [32]:
Municípios_Estados = pd.read_excel(r'C:\Projeto Final\data\RELATORIO_DTB_BRASIL_MUNICIPIO.xls')

somente as colunas que irão ser utilizadas para a análise

In [38]:
Municípios_Estados['Região Geográfica Imediata'].unique()

array([110005, 110002, 110004, 110006, 110001, 110003, 120001, 120002,
       120004, 120005, 120003, 130005, 130006, 130004, 130003, 130009,
       130001, 130002, 130010, 130008, 130007, 130011, 140001, 140002,
       140004, 140003, 150003, 150009, 150001, 150020, 150014, 150015,
       150019, 150018, 150005, 150007, 150016, 150011, 150012, 150006,
       150021, 150002, 150010, 150008, 150004, 150017, 150013, 160003,
       160004, 160001, 160002, 170003, 170008, 170010, 170011, 170005,
       170001, 170009, 170007, 170006, 170002, 170004, 210021, 210013,
       210007, 210001, 210010, 210009, 210022, 210011, 210019, 210004,
       210003, 210008, 210020, 210005, 210002, 210017, 210006, 210012,
       210014, 210015, 210018, 210016, 220010, 220002, 220009, 220001,
       220016, 220013, 220018, 220004, 220017, 220003, 220015, 220005,
       220008, 220012, 220006, 220007, 220019, 220011, 220014, 230012,
       230003, 230017, 230009, 230011, 230015, 230007, 230002, 230005,
      

In [40]:
Mortalidade_2019['CODMUNOCOR'].unique()

array([290750, 421900, 420460, ..., 520120, 354180, 315380], dtype=int64)

In [42]:
Municípios_Estados[Municípios_Estados['Região Geográfica Imediata']==290750]

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município


In [30]:
Municípios_Estados = Municípios_Estados.loc[:, ['Nome_UF', 'Região Geográfica Imediata']]

In [31]:
Municípios_Estados

,Nome_UF,Região Geográfica Imediata
0,Rondônia,110005
1,Rondônia,110005
2,Rondônia,110002
3,Rondônia,110004
4,Rondônia,110002
...,...,...
5565,Goiás,520002
5566,Goiás,520009
5567,Goiás,520022
5568,Goiás,520018


# Juntando as Tabelas

In [28]:
mortalidade_UF_2019 = pd.merge(Mortalidade_2019, Municípios_Estados, left_on = 'CODMUNOCOR', right_on ='Região Geográfica Imediata')

In [29]:
mortalidade_UF_2019

,DTOBITO,CODMUNOCOR,DTNASC,IDADE,SEXO,RACACOR,CAUSABAS,Classificação de causas,Nome_UF,Região Geográfica Imediata
0,2019-01-29,350050,1970-01-01,41.0,1,1.0,I21,Doença,São Paulo,350050
1,2019-01-29,350050,1970-01-01,41.0,1,1.0,I21,Doença,São Paulo,350050
2,2019-01-29,350050,1970-01-01,41.0,1,1.0,I21,Doença,São Paulo,350050
3,2019-01-29,350050,1970-01-01,41.0,1,1.0,I21,Doença,São Paulo,350050
4,2019-01-29,350050,1970-01-01,41.0,1,1.0,I21,Doença,São Paulo,350050
...,...,...,...,...,...,...,...,...,...,...
87261,2019-07-12,420005,1970-01-01,84.0,2,1.0,I10,Doença,Santa Catarina,420005
87262,2019-07-12,420005,1970-01-01,84.0,2,1.0,I10,Doença,Santa Catarina,420005
87263,2019-07-12,420005,1970-01-01,84.0,2,1.0,I10,Doença,Santa Catarina,420005
87264,2019-07-12,420005,1970-01-01,84.0,2,1.0,I10,Doença,Santa Catarina,420005


In [45]:
Município_estabelecimento = pd.read_csv(r'C:\Projeto Final\data\cadastro_estabelecimentos_cnes.csv', sep = ';')

In [48]:
Município_estabelecimento = Município_estabelecimento.loc[:,['UF', 'IBGE']]

In [49]:
Município_estabelecimento

,UF,IBGE
0,52,520170
1,26,260290
2,26,260290
3,26,260290
4,26,260290
...,...,...
42597,35,355030
42598,35,355030
42599,35,355030
42600,35,355030


In [50]:
mortalidade_UF_2019 = pd.merge(Mortalidade_2019, Município_estabelecimento, left_on = 'CODMUNOCOR', right_on ='IBGE')

In [51]:
mortalidade_UF_2019

,DTOBITO,CODMUNOCOR,DTNASC,IDADE,SEXO,RACACOR,CAUSABAS,Classificação de causas,UF,IBGE
0,2019-01-21,290750,1970-01-01,57.0,1,2.0,I12,Doença,29,290750
1,2019-01-21,290750,1970-01-01,57.0,1,2.0,I12,Doença,29,290750
2,2019-01-21,290750,1970-01-01,57.0,1,2.0,I12,Doença,29,290750
3,2019-01-21,290750,1970-01-01,57.0,1,2.0,I12,Doença,29,290750
4,2019-01-21,290750,1970-01-01,57.0,1,2.0,I12,Doença,29,290750
...,...,...,...,...,...,...,...,...,...,...
138406249,2019-12-23,520120,1970-01-01,64.0,1,1.0,E14,Doença,52,520120
138406250,2019-02-26,354180,1970-01-01,62.0,2,4.0,J45,Doença,35,354180
138406251,2019-02-09,354180,1970-01-01,58.0,1,1.0,I21,Doença,35,354180
138406252,2019-06-23,315380,1970-01-01,53.0,1,1.0,V49,Causas externas de morbidade e de mortalidade ...,31,315380


In [60]:
x = mortalidade_UF_2019.drop_duplicates

In [61]:
x

<bound method DataFrame.drop_duplicates of              DTOBITO  CODMUNOCOR     DTNASC  IDADE  SEXO  RACACOR CAUSABAS  \
0         2019-01-21      290750 1970-01-01   57.0     1      2.0      I12   
1         2019-01-21      290750 1970-01-01   57.0     1      2.0      I12   
2         2019-01-21      290750 1970-01-01   57.0     1      2.0      I12   
3         2019-01-21      290750 1970-01-01   57.0     1      2.0      I12   
4         2019-01-21      290750 1970-01-01   57.0     1      2.0      I12   
...              ...         ...        ...    ...   ...      ...      ...   
138406249 2019-12-23      520120 1970-01-01   64.0     1      1.0      E14   
138406250 2019-02-26      354180 1970-01-01   62.0     2      4.0      J45   
138406251 2019-02-09      354180 1970-01-01   58.0     1      1.0      I21   
138406252 2019-06-23      315380 1970-01-01   53.0     1      1.0      V49   
138406253 2019-08-07      315380 1970-01-01   80.0     2      4.0      I10   

                    